In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.make'):
    shutil.rmtree('./file/3.make')

os.makedirs('./file/3.make')

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1

    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error, key

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                if len(json.loads(input_.loc[a, 'Json_Part Code'])) != 1:
                    raise
                
                part_code = json.loads(input_.loc[a, 'Json_Part Code'])['0']
                
                data = {'getArticleLinkedAllLinkingTargetManufacturer2': {'articleCountry': 'DE',
                                                                          'articleId': part_code,
                                                                          'country': 'DE',
                                                                          'countryGroupFlag': False,
                                                                          'linkingTargetType': 'VOLB',
                                                                          'provider': 40_000}}
                
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                             data=json.dumps(data),
                                             headers=get_header(**{'x-api-key': key}),
                                             #proxies=get_proxy(),
                                             timeout=(10, 10))
                            
                        if resp.status_code == 200:
                            json_ = resp.json()
                
                            if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                                while True:
                                    try:
                                        data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                                     'username': '186299u15',
                                                                     'password': 'APMTeDoW14'}}
                                        
                                        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                                             data=json.dumps(data),
                                                             headers=get_header(),
                                                             #proxies=get_proxy(),
                                                             timeout=(10, 10))
                                
                                        if resp.status_code == 200:
                                            json_ = resp.json()
                                
                                            if json_['status'] == 200:
                                                key = json_['apiKey']
                                                break
                                    except KeyboardInterrupt:
                                        break
                                    except:
                                        continue
                            elif json_['status'] == 200:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_make = json_['data']['array']

                # = = = = = = = = = = = = = = =

                if not list_make:
                    raise

                # = = = = = = = = = = = = = = =

                list_make_code = [make['manuId'] for make in list_make]

                # = = = = = = = = = = = = = = =

                list_make = [make['manuName'].strip() for make in list_make]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                        'Part Number': input_.loc[a, 'Part Number'],
                                        'Part Code': part_code,
                                        'Link Type': 'VOLB',
                                        'Make No': [i+1 for i in range(len(list_make))],
                                        'Make': list_make,
                                        'Make Code': list_make_code})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(50):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Make No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.make/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-make_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.make'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.make/{file}',
              f'''./file/3.make/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：96

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 128.07649
[剩余数量：46] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 339.104447
[剩余数量：45] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 18.01953
[剩余数量：44] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 776.17446
[剩余数量：43] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 194.10194
[剩余数量：42] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 62.04692
[剩余数量：41] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 94.06790
[剩余数量：40] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 84.05865
[剩余数量：39] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 849.175774
[剩余数量：38] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 82.05863
[剩余数量：37] - [当前时间：16:01:42]

[状态：error，尝试次数：1] - 50.00% - crawler_1-make_error > 667.172199
[剩余数量：36] - [当前时间：16:0

Progress: 0it [00:00, ?it/s]

Done ~
